In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

spark = SparkSession.builder \
                    .appName('Read and Write') \
                    .config('spark.jars.packages', 'mysql:mysql-connector-java:8.0.23') \
                    .getOrCreate()

spark

2021-09-22 01:39:10,468 WARN util.Utils: Your hostname, localhost resolves to a loopback address: 127.0.0.1; using 192.168.64.2 instead (on interface eth0)
2021-09-22 01:39:10,470 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.1.2-bin-without-hadoop/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
mysql#mysql-connector-java added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b2f9c91c-749b-45fb-b90d-fb91dc55101e;1.0
	confs: [default]
	found mysql#mysql-connector-java;8.0.23 in central
	found com.google.protobuf#protobuf-java;3.11.4 in central
downloading https://repo1.maven.org/maven2/mysql/mysql-connector-java/8.0.23/mysql-connector-java-8.0.23.jar ...
	[SUCCESSFUL ] mysql#mysql-connector-java;8.0.23!mysql-connector-java.jar (40531ms)
downloading https://repo1.maven.org/maven2/com/google/protobuf/protobuf-java/3.11.4/protobuf-java-3.11.4.jar ...
	[SUCCESSFUL ] com.google.protobuf#protobuf-java;3.11.4!protobuf-java.jar(bundle) (37521ms)
:: resolution report :: resolve 8548ms :: artifacts dl 78079ms
	:: modules in use:
	com.google.protobuf#protobuf-java;3.11.4 from central in [default]
	mysql#mysql-connector-java;8.0.23 from central in [default]
	--

## CSV

In [ ]:
csv_df = spark.read \
              .options(delimeter=',') \
              .csv('../datasets/forestfires.csv', header=True, inferSchema=True)
csv_df.show()
csv_df.count()

In [ ]:
csv_df.write.csv('../datasets/write-csv', mode='overwrite')

## Parquet

In [ ]:
parquet_df = spark.read \
                  .parquet('../datasets/olympic-history/athletes_events')
parquet_df.show()
parquet_df.count()

In [ ]:
parquet_df.write.parquet('../datasets/write-parquet', mode='overwrite')

## Database

In [ ]:
# Setup the JDBC connection
jdbc_url = "jdbc:mysql://localhost:3306/loja"
connection_properties = {
      "user" : "root",
      "password" : "123",
      "driver" : "com.mysql.jdbc.Driver"
    }

# Create a query
query = """
            SELECT * FROM bairro
        """

# run the query
bh_bairros_df = spark.read \
                     .jdbc(url=jdbc_url, 
                           table=f"({query}) AS t", 
                           properties=connection_properties)

bh_bairros_df.show()
bh_bairros_df.count()
bh_bairros_df.printSchema()

In [ ]:
from pyspark.sql import functions as F

bh_bairros_df.write \
             .mode("overwrite") \
             .option("truncate", "true") \
             .jdbc(url=jdbc_url, 
                   table="bairro",
                   properties=connection_properties)

bh_bairros_df.printSchema()